In [25]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id j-Y8H7E819F8SD --auth-type None  

The sagemaker_studio_analytics_extension.magics extension is already loaded. To reload it, use:
  %reload_ext sagemaker_studio_analytics_extension.magics
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1650067460964_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-Y8H7E819F8SD", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


#### 注意事项

您需要访问设置为使用 Amazon Virtual Private Cloud (Amazon VPC) 模式的 SageMaker Studio。

SageMaker Studio 使用的所有子网必须是私有子网。

如果您将sm-analytics实用程序来配置 SparkMagic 内核，请遵循以下两个先决条件之一：

- 确保 Amazon VPC 接口终端节点已连接到 SageMaker Studio 使用的所有子网。

- 确保 SageMaker Studio 使用的所有子网都路由为使用 NAT 网关。有关更多信息，请参阅 。NAT 网关.

如果以下任一点适用于您，则在使用亚马逊 EMR 时必须安装 Spark 和 Livy。

- 您的亚马逊 EMR 集群与 Studio 位于同一亚马逊 VPC 中。

- 您的集群位于连接到 Studio 中的 Amazon VPC 中的亚马逊 VPC 中。

Amazon SageMaker Studio 和亚马逊 EMR 的安全组必须允许相互访问和相互访问。

您的亚马逊 EMR 安全组必须打开端口 8998，以便 Amazon SageMaker Studio 可以通过 Livy 与 Spark 集群进行通信。

In [26]:
from pyspark.sql.session import SparkSession

spark = SparkSession(sc)
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 处理原始数据 

将S3导出数据进行处理


In [28]:
df=spark.read.csv("s3://anti-fraud-workshop/glue-data/", header=True)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----+--------------------+---+-----+-----+----+--------------------+--------+--------------------+------------+-----+--------------------+-----+-----+
| symbol|      counter_userid|side|            order_id|fee|index|label|done|             user_id|fee_rate|     counter_orderid|        size|price|            trade_id|funds|   id|
+-------+--------------------+----+--------------------+---+-----+-----+----+--------------------+--------+--------------------+------------+-----+--------------------+-----+-----+
|DAI-BTC|6c3e226b4d4795d51...|null|28c73e9fd11424b7d...|0.0| 1665|    0|   1|0c0d62bb537b55ecb...|     0.0|6c3e226b4d4795d51...|   1422832.5|  0.0|68748a9290249902c...|  0.0| 1666|
|DAI-BTC|6c3e226b4d4795d51...|null|ab12772f2f61f8e9b...|0.0| 8717|    0|   1|d162133ebabf9955d...|     0.0|6c3e226b4d4795d51...| 7238881.914|  0.0|fd76350e39119493d...|  0.0| 8718|
|DAI-BTC|6c3e226b4d4795d51...|null|0acdd1f56a3e8411e...|0.0|19479|    0|   1|d162133ebabf9955d.

In [37]:
df.drop("symbol","trade_id","counter_userid","order_id","counter_orderid","id","index")
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---+-----+----+--------------------+--------+------------+-----+-----+
|side|fee|label|done|             user_id|fee_rate|        size|price|funds|
+----+---+-----+----+--------------------+--------+------------+-----+-----+
|null|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|  0.0|
|null|0.0|    0|   1|d162133ebabf9955d...|     0.0| 7238881.914|  0.0|  0.0|
|null|0.0|    0|   1|d162133ebabf9955d...|     0.0|3342742.6806|  0.0|  0.0|
|null|0.0|    0|   1|a7920e0622c5f5ce6...|     0.0| 203408.1342|  0.0|  0.0|
|null|0.0|    0|   1|d162133ebabf9955d...|     0.0| 7238881.914|  0.0|  0.0|
|null|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|  0.0|
|null|0.0|    0|   1|d162133ebabf9955d...|     0.0|3177205.4202|  0.0|  0.0|
|null|0.0|    0|   1|d9eaaa14faf09ec1c...|     0.0|1525665.3957|  0.0|  0.0|
|null|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|  0.0|
|null|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|  0.0|

In [39]:
# 转化side列
from pyspark.sql.functions import col, create_map, lit
from itertools import chain

mapping = {'buy': 2, 'sell': 1}

mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])

df = df.withColumn("side_process", mapping_expr.getItem(col("side")))

df = df.fillna(0, subset=['side_process'])

df = df.drop('side')

df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+----+--------------------+--------+------------+-----+-----+------------+
|fee|label|done|             user_id|fee_rate|        size|price|funds|side_process|
+---+-----+----+--------------------+--------+------------+-----+-----+------------+
|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|  0.0|           0|
|0.0|    0|   1|d162133ebabf9955d...|     0.0| 7238881.914|  0.0|  0.0|           0|
|0.0|    0|   1|d162133ebabf9955d...|     0.0|3342742.6806|  0.0|  0.0|           0|
|0.0|    0|   1|a7920e0622c5f5ce6...|     0.0| 203408.1342|  0.0|  0.0|           0|
|0.0|    0|   1|d162133ebabf9955d...|     0.0| 7238881.914|  0.0|  0.0|           0|
|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|  0.0|           0|
|0.0|    0|   1|d162133ebabf9955d...|     0.0|3177205.4202|  0.0|  0.0|           0|
|0.0|    0|   1|d9eaaa14faf09ec1c...|     0.0|1525665.3957|  0.0|  0.0|           0|
|0.0|    0|   1|0c0d62bb537b55ecb...|     0.0|   1422832.5|  0.0|

In [18]:
# 切分数据集
train_df, test_df = data.randomSplit([0.9, 0.1], 42)
train_df, val_df = train_df.randomSplit([0.8, 0.2], 42)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# 删除user_id列，训练数据不需要这一列
train_df = train_df.drop('user_id')
val_df = val_df.drop('user_id')

In [33]:
# shuffle data
from pyspark.sql.functions import rand

def shuffle_df(df):
    df_with_rand = df.withColumn('rand', rand(seed=42)).orderBy('rand')
    df_shuffle = df_with_rand.drop('rand')
    return df_shuffle

train_df_shuffle = shuffle_df(train_df)
val_df_shuffle = shuffle_df(val_df)
test_df_shuffle = shuffle_df(test_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# 上传至s3
train_df_shuffle.coalesce(1).write.csv("s3://sagemaker-us-east-1-851108988172/detect-workshop/data_processed/xgboost_train")
val_df_shuffle.coalesce(1).write.csv("s3://sagemaker-us-east-1-851108988172/detect-workshop/data_processed/xgboost_val")
test_df_shuffle.coalesce(1).write.csv("s3://sagemaker-us-east-1-851108988172/detect-workshop/data_processed/xgboost_test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…